In [2]:
#Import modules
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.dates as dt
import pandas as pd
import math

In [3]:
#===========Define Key Functions===============================================
def generate_file_list(data_dir, fname, numlines = 0):
    
    '''This function opens a text file containing a list of filenames to process,
    reads the lines, appends the directory path to the start of each filename, 
    and returns the data as a list of stings with a full-path to each file to
    be processed.'''
    
    #Open a file object
    file_var = open(data_dir + fname, 'r')
    
    # Call the readlines method of the file object to read the data from the 
    # file object, where each line of the file is a new item in the list. The 
    # readlines method returns a list of strings. 
    file_list = file_var.readlines()
    
    # Remove the newline character and append the directory path to the start 
    # of each filename
    for line in range(len(file_list)):
        
        file_list[line] = file_list[line].rstrip()
        file_list[line] = data_dir + file_list[line]
    
    return file_list[:numlines]
    
def netcdf_time_scale(y,m,d,h,y0):
    
    '''This function takes a date in y,m,d,h format and returns the value of the
    hours since jan 1, y0 (the time convention of reanalysis). Inputs are integers. 
    Output is a float.''' 
    
    d0 = datetime(y0,1,1,0) # the datetime object representing 12am, jan 1 of the baseline year of the reanalysis
    d = datetime(y,m,d,h) # the datetime object for the day and time being tested
    
    t = 24 * (dt.date2num(d) - dt.date2num(d0)) # number of hours since 12am, jan 1 of the baseline year
    # dt.date2num gives the number of days since the python datetime time origin. 
    
    return t

def date_from_netcdftime(t_netcdf, y0):
    
    '''This function takes a datetime64 object and converts it to a date in y,m,d,h format. Output is a tuple. '''
    '''
    d0 = datetime(y0,1,1,0) # the datetime object representing 12am, jan 1 of the baseline year of the reanalysis
    
    d = dt.date2num(t_netcdf) / 24 + dt.date2num(d0) # number of days since jan 1 of the python datetime baseline year
    
    date = dt.num2date(d)
    '''
    date = pd.Timestamp(t_netcdf)
    
    y = date.year
    m = date.month
    d = date.day
    h = date.hour
    
    return y,m,d,h

def horizontal_distance(low1, low2):
    
    '''This function takes a pair of (lat,lon) coordinates, and computes the linear
    distance between them. Output is a float. '''
    
    lat1 = low1[0]
    lat2 = low2[0]
    
    lon1 = low1[1]
    lon2 = low2[1]
    
    lat_distance = abs(lat2 - lat1)
    
    # This part handles the situation where the two lows are close on either
    # side of the line of 0 degrees longitude. 
    lon_dist1 = abs(lon2 - lon1)
    lon_dist2 = 360 - abs(lon2 - lon1)
    lon_distance = np.minimum(lon_dist1, lon_dist2) # np.minimum is important
    # here so this function can be vectorised across a whole array of low centers.
    
    distance = (lat_distance ** 2 + lon_distance ** 2) ** 0.5
    
    return distance
    
def  low_center_test(field, threshold, a, b):
    
    '''This function takes as its input an array of numbers, and tests if the
    cell [y,x] is the center of a local low by a minimum threshold. The cell
    [y,x] is identified as a low center if its value is lower than the value of
    each of the surrounding 8 cells at a distance of 2.5 degrees lat/lon, as well
    as lower than the value of each of the surrounding 8 cells at a distance of
    5 degrees lat/lon by a minimum difference threshold. The diagonal difference 
    checks are scaled by a factor of 2**0.5 by Pythagoras. The output is a tuple
    of a boolean of whether the cell passes the low test or not, as well as 
    the values of the central cell value and the test 1 and test 2 mean values.'''
    
    # Get the center of an arbitrarily-shaped box
    x = int((np.shape(field)[0] - 1) / 2)
    y = x # center of a square has the same x and y value
    
    CELL = field[y,x] # gets the value of the field in the center of the box
    
    #Comparison between the central cell and the surrounding 8 cells
    TEST1 = np.array([field[y-a,x-a]-CELL,field[y-a,x]-CELL,field[y-a,x+a]-CELL,field[y,x+a]-CELL,field[y+a,x+a]-CELL,field[y+a,x]-CELL,field[y+a,x-a]-CELL,field[y,x-a]-CELL],float)
    TEST1[0::2] = TEST1[0::2]/(2**0.5)
    
    TEST2 = np.array([field[y-b,x-b]-CELL,field[y-b,x]-CELL,field[y-b,x+b]-CELL,field[y,x+b]-CELL,field[y+b,x+b]-CELL,field[y+b,x]-CELL,field[y+b,x-b]-CELL,field[y,x-b]-CELL],float)
    TEST2[0::2] = TEST2[0::2]/(2**0.5)
    
    if (np.all(TEST1 >= 0)) and (np.all(TEST2 >= threshold)):
    #if (np.all(TEST1 >= 0)) and (np.mean(TEST2) >= threshold):
             
        a = True
        b = CELL
        c = np.mean(TEST1)
        d = np.mean(TEST2)
   
    else:
        
        a = False
        b = np.nan
        c = np.nan
        d = np.nan
        
    return a, b, c, d

def eliminate_secondary_lows(low_array, distance_threshold):
    
    '''This function removes minor low centers that are close to a deeper parent
    low center in the same field. It does this by looping through each low center
    identified in a field, and comparing it to each other low center in the same
    field. If the two lows are within a given distance threshold, the shallower
    low is overwritten with np.nan. Only the non-nan lows are then returned out
    of the function. Output is a numpy array.'''
    
    # creates an array of each timestep with at least one low to loop through.
    timerange = np.unique(low_array[:,5]).astype(int)
    #print(timerange)
    
    for tstep in timerange: 
        
        #print(np.shape(low_array[(low_array[:,5] == tstep), :]))
        #print(tstep)
        #if np.shape(low_array[(low_array[:,5] == tstep), :])[0] == 0:
            #print('******==========ERROR IS HERE==========******')
            #print(tstep)
        
        # extracts all of the rows in the low_array for the timestep being tested
        ifield = (low_array[:,5] == tstep)
        fieldlows = low_array[ifield, :]
        
        arraylen = np.shape(fieldlows)[0]
        arrayloop = np.arange(0,arraylen,1)

        if arraylen > 0:
            
            # Just a check to test if the deepest low entry has been blanked out. 
            # Will be compared to postfiltermin after the filtering process. 
            prefiltermin = np.nanmin(fieldlows[:,8])
            
            for a in arrayloop:
                
                low1 = fieldlows[a,:]
                array_subloop = np.arange(a + 1, arraylen, 1)
                
                for b in array_subloop:
                    
                    low2 = fieldlows[b,:]
                    
                    # These conditional tests indicate whether two closed low 
                    # instances are within the spatial proximity of each other, and
                    # which low center is the deepest. 
                    distance_test = horizontal_distance((low1[6], low1[7]), (low2[6], low2[7])) <= distance_threshold 
                    low1_shallower = (low1[8] > low2[8])
                    low2_shallower = (low2[8] > low1[8])
                    equal_lowtest = (low2[8] == low1[8])
                    
                    if distance_test and low1_shallower:
                        
                        low1[:] = np.nan
                        
                    elif distance_test and low2_shallower:
                        
                        low2[:] = np.nan
                        
                    elif distance_test and equal_lowtest:
                        
                        if low1[10] < low2[10]:
                            
                            low1[:] = np.nan
                            
                        elif low2[10] < low1[10]:
                            
                            low2[:] = np.nan
                                         
                        #else:
                        
                            # It may be helpful to receive an allert when two low
                            # centers of equal depth are located in the array (which)
                            # does occasionally happen. In this case both lows are
                            # retained, though only one may persist through the tracking
                            # stage. 
                            #print('FOUND TWO LOW CENTERS OF EQUAL DEPTH IN FIELD.')
                            #print(low1)
                            #print(low2)
                    
            postfiltermin = np.nanmin(fieldlows[:,8])
            
            if prefiltermin != postfiltermin:
                
                # This code checks if the deepest low in a timestep has been removed,
                # due to some error in the process. It prints out the date of the
                # data causing the error to help with diagnosing the cause of the error. 
                errordate = date_from_netcdftime(tstep, 1800)
                print('Filtering error at tstep: ', errordate)
                print('Prefiltermin: ', prefiltermin)
                print('Postfiltermin: ', postfiltermin, '\n')
                print('fieldlows: ', fieldlows, '\n')
            
            # The filtered low data (including np.nan rows) are written back into 
            # the base low array. 
            low_array[ifield,:] = fieldlows
    
    # This stage removes all the rows from the array that were blanked out with
    # np.nan. 
    ir = np.isnan(low_array[:,0])
    refined_lows = low_array[(ir == False),:]
    
    return refined_lows


In [4]:
#==========SECTION 1 Identification Scheme Parameters===========================

dataset = 'eraint' # ncep1 or eraint or era5

save_path = '/home/561/nxg561/00_Tracking_Scheme_Comparison/Input_Data/Nick_Lows/'

#Time period setup
year_range = (1979, 2017) #(start year,end year)
startdate = (1,1) #(month,day)
enddate = (12,31) #(month,day)

#Spatial domain setup
lat_range = (-21,-60) #Boundary of region in latitude
lon_range = (-180, 179) #(-180,179) for ERA-Int, (-180, 178.5) for ERA5 #Boundary of region in longitude

p_level = 50000 # 500 hPa for ERA5 and NCEP1, 50000 Pa for ERA-Int.  

# Reanalysis data properties
t_res = 6 #hrs
spacial_res = 1 #degrees lat/lon # 1.5 for ERA5, 1 for ERA-Int

#Cutoff low properties setup
hgt_threshold = 0 #gpm. The minimum difference in pressure between the center
            #square and each of the surrounding 8 squares to identify a closed 
            #low.
            
pg_threshold = 0  #hPa. Similar as above.

#Filtering parameters

filter_minor_lows = True

low_filtering_spatial_threshold = 10 #For removing weaker low centers which are close to a 
            #deeper low center in the SAME pressure field. 
            #The maximum distance (in degees lat/lon) between two low centers 
            #in the same pressure field to consider them the same system.

# THESE VARIABLES ARE ONLY USED FOR THE TRACKING STAGE. 
track_joining_spatial_threshold = 7.5 #For joining CONSECUTIVE low centers into tracks. The maximum 
            #distance (in degees lat/lon) between one low center in the 
            #'current' pressure field, and another low center in previous
            #pressure fields to join them to the same track. 
            
minimum_duration = 0 #hrs. The minimum duration for a low event to be included
            # in the dataset, to eliminate short insignificant lows. 

if dataset == 'eraint':
    
    data_path = '/g/data/w40/nxg561/ERA-Int/'
    hgt_file_prefix = 'z500.'
    slp_file_prefix = 'slp.'
    file_extension = '_rgd.nc'

    time_y0 = 1900
    
    lat_variable_name = 'lat'
    lon_variable_name = 'lon'
    vertical_coordinate_name = 'lev' # 'level' for NCEP1, 'lev' for ERAInterim
    time_variable_name = 'time'
    gph_variable_name = 'z' # 'hgt' for NCEP1, 'z' for ERAInterim
    slp_variable_name = 'psl'
    
elif dataset == 'ncep1':
    
    data_path = '/Volumes/ExternalHD/Large_Data/NCEP1/'
    hgt_file_prefix = 'hgt.'
    slp_file_prefix = 'slp.'
    file_extension = '.nc'

    time_y0 = 1800
    
    lat_variable_name = 'lat'
    lon_variable_name = 'lon'
    vertical_coordinate_name = 'level' # 'level' for NCEP1, 'lev' for ERAInterim
    time_variable_name = 'time'
    gph_variable_name = 'hgt' # 'hgt' for NCEP1, 'z' for ERAInterim
    slp_variable_name = 'slp'
    
if dataset == 'era5':
    
    data_path = '/g/data/w40/nxg561/ERA5/'#'/Volumes/ExternalHD/Large_Data/ERA-Interim/Regridded/'
    #file_list = 'era5_file_list_z_' + str(year_range[0]) + '.txt'

    time_y0 = 1900
    
    lat_variable_name = 'lat'
    lon_variable_name = 'lon'
    vertical_coordinate_name = 'level' # 'level' for NCEP1, 'lev' for ERAInterim
    time_variable_name = 'time'
    gph_variable_name = 'z' # 'hgt' for NCEP1, 'z' for ERAInterim
    slp_variable_name = 'psl'

In [5]:
#==========SECTION 2 Utility variable setup=====================================

#this section prealocates an array of zeros to store the data of the identified 
#lows in, with a number of rows for every time step in the whole time period.
    
tsteps = (netcdf_time_scale(year_range[1],12, 31, 18, time_y0) - netcdf_time_scale(year_range[0], 1, 1, 0, time_y0) ) + 1

raw_lows_z = np.zeros((500 * int(tsteps), 11), dtype = float) #prealocates an array of 
            #zeros to store the lows in. I just guessed 500 x the total number of 
            #timesteps would be enough rows.

# This part computes the conversion factor between the resolution of ncep1 (the 
# original dataset that this code was developed for) and the resolution of the 
# dataset being used here. 
scale_factor = 2.5 / spacial_res
    
a = math.floor(1 * scale_factor) # grid spaces for 1 grid space equivalent
b = math.floor(2 * scale_factor) # grid spaces for 1 grid space equivalent

In [6]:
#==========SECTION 3 Data Input and Low Detection===============================

# This part creates a list of filenames to open as a mf_dataset in xarray:

#years = tuple(range(year_range[0],year_range[1]+1))

#file_list = generate_file_list(data_path, file_list, numlines = 12)
# Load all the reanalysis files in a multi-file dataset, and find all the locations
# of local minima:

#xr_obj = xr.open_mfdataset(file_list, combine = 'nested', concat_dim = 'time', engine = 'netcdf4')
xr_obj = xr.open_mfdataset(data_path + 'z*.nc', combine = 'nested', concat_dim = 'time', engine = 'netcdf4')
# Extract the data for the search domain, as well as the domain shifted one and
# two grid spaces to the north and south. The east and west shifted data will
# be obtained with a np.roll() function. These variables are an xarray data array. 
hgt_data = xr_obj[gph_variable_name].sel(lat = slice (lat_range[0], lat_range[1]), lon = slice (lon_range[0], lon_range[1]), lev = p_level)

hgt_data_north = xr_obj[gph_variable_name].sel(lat = slice (lat_range[0] + a * spacial_res, lat_range[1] + a * spacial_res), lon = slice (lon_range[0], lon_range[1]), lev = p_level)
hgt_data_north2 = xr_obj[gph_variable_name].sel(lat = slice (lat_range[0] + b * spacial_res, lat_range[1] + b * spacial_res), lon = slice (lon_range[0], lon_range[1]), lev = p_level)

hgt_data_south = xr_obj[gph_variable_name].sel(lat = slice (lat_range[0] - a * spacial_res, lat_range[1] - a * spacial_res), lon = slice (lon_range[0], lon_range[1]), lev = p_level)
hgt_data_south2 = xr_obj[gph_variable_name].sel(lat = slice (lat_range[0] - b * spacial_res, lat_range[1] - b * spacial_res), lon = slice (lon_range[0], lon_range[1]), lev = p_level)

# Extract the numpy arrays to work with the actual data
hgt_field = hgt_data.values
hgt_field_north = hgt_data_north.values
hgt_field_north2 = hgt_data_north2.values
hgt_field_south = hgt_data_south.values
hgt_field_south2 = hgt_data_south2.values

# Some reanalysis datasets need to have geopotential height divided by g = 9.8 m/s^2. 
if dataset == 'era5' or dataset == 'eraint':
        
        hgt_field = hgt_field / 9.8
        hgt_field_north = hgt_field_north / 9.8
        hgt_field_north2 = hgt_field_north2 / 9.8
        hgt_field_south = hgt_field_south / 9.8
        hgt_field_south2 = hgt_field_south2 / 9.8
        
# Do the rolls to set up the east, west, north-east, north-west, south-east and
# south-west shifted data here        
hgt_field_west = np.roll(hgt_field, a, axis = 2)
hgt_field_east = np.roll(hgt_field, -a, axis = 2)

hgt_field_northeast = np.roll(hgt_field_north, -a, axis = 2)
hgt_field_northwest = np.roll(hgt_field_north, a, axis = 2)
hgt_field_southeast = np.roll(hgt_field_south, -a, axis = 2)
hgt_field_southwest = np.roll(hgt_field_south, a, axis = 2)

hgt_field_west2 = np.roll(hgt_field, b, axis = 2)
hgt_field_east2 = np.roll(hgt_field, -b, axis = 2)

hgt_field_northeast2 = np.roll(hgt_field_north2, -b, axis = 2)
hgt_field_northwest2 = np.roll(hgt_field_north2, b, axis = 2)
hgt_field_southeast2 = np.roll(hgt_field_south2, -b, axis = 2)
hgt_field_southwest2 = np.roll(hgt_field_south2, b, axis = 2)

# This is where the local minima are actually found:      
t1n = hgt_field_north - hgt_field
t1ne = (hgt_field_northeast - hgt_field) / (2 ** 0.5)
t1e = hgt_field_east - hgt_field
t1se = (hgt_field_southeast - hgt_field) / (2 ** 0.5)
t1s = hgt_field_south - hgt_field
t1sw = (hgt_field_southwest - hgt_field) / (2 ** 0.5)
t1w = hgt_field_west - hgt_field
t1nw = (hgt_field_northwest - hgt_field) / (2 ** 0.5)

t2n = hgt_field_north2 - hgt_field
t2ne = (hgt_field_northeast2 - hgt_field) / (2 ** 0.5)
t2e = hgt_field_east2 - hgt_field
t2se = (hgt_field_southeast2 - hgt_field) / (2 ** 0.5)
t2s = hgt_field_south2 - hgt_field
t2sw = (hgt_field_southwest2 - hgt_field) / (2 ** 0.5)
t2w = hgt_field_west2 - hgt_field
t2nw = (hgt_field_northwest2 - hgt_field) / (2 ** 0.5)

#low_test is a numpy array of True/False values, where a True value corresponds
# to a point in the geopotential height field that is lower than each of the 16
# surrounding points. The shape of the low_test array corresponds to the shape
# of the hgt_field (i.e. time, lat, lon.)
low_test = (t1n > 0) & (t1ne > 0) & (t1e > 0) & (t1se > 0) & (t1s > 0) & (t1sw > 0) & (t1w > 0) & (t1nw > 0) & (t2n > hgt_threshold) & (t2ne > hgt_threshold) & (t2e > hgt_threshold) & (t2se > hgt_threshold) & (t2s > hgt_threshold) & (t2sw > hgt_threshold) & (t2w > hgt_threshold) & (t2nw > hgt_threshold)

# Now we loop through each of the indentified minima from the previous
# step, pull out the 5*5 gridsquare field from the global geopotential
# height array, and compute the cyclone metrics with the functions
# defined at the top of the program. 

# It is important to extract a new field of hgt_data covering the whole globe, 
# to handle local minima identidfied at the edges of the study domain.                        
hgt_global_da = xr_obj[gph_variable_name].sel(lev = p_level)
hgt_data_global = hgt_global_da.values

domain_lats = hgt_global_da['lat'].values
domain_lons = hgt_global_da['lon'].values
domain_time = hgt_global_da['time'].values

lat_inds_global = tuple(np.squeeze(np.where((domain_lats <= lat_range[0]) & (domain_lats >= lat_range[1]))))
lat_offset = lat_inds_global[0] # This accounts for the fact that the row index
# identified in the variable 'local_minima' refers to the subset region
# of the specific band of latitudes being studied. 

# cyclone_inds is a tuple of three arrays consisting of the time, lat and lon
# indices of each of the 'True' values in low_test.
cyclone_inds = np.where(low_test)

cyclone_tsteps = cyclone_inds[0][:]
cyclone_lats = cyclone_inds[1][:]
cyclone_lons = cyclone_inds[2][:]
    
local_minima_loop = range(0,len(cyclone_inds[0]), 1)

low_center_count = 0 

for l in local_minima_loop:
    
    lat = cyclone_lats[l] + lat_offset
    lon = cyclone_lons[l]
    t = cyclone_tsteps[l]
    
    #print('tstep,lat,lon: ',t , ',', lat, ',', lon)
    
    # These next tests handle cases of a closed low found where it overlaps the 
    # International Dateline
    if lon < b:
        
        # Extract the cyclone_field in two parts and concatenate them here
        west = hgt_data_global[t, lat - b : lat + b + 1, len(domain_lons) - (b - lon) : ]
        east = hgt_data_global[t, lat - b : lat + b + 1, : b + lon + 1]
        cyclone_field = np.concatenate((west, east), axis = 1)
        
    elif lon >= len(domain_lons) - b:
        
        inv_lon = len(domain_lons) - lon
        west = hgt_data_global[t, lat - b : lat + b + 1, len(domain_lons) - (b + inv_lon) : ]
        east = hgt_data_global[t, lat - b : lat + b + 1, : b - inv_lon + 1]
        cyclone_field = np.concatenate((west, east), axis = 1)
    
    # This last case is all other lows in the middle of the domain.
    else:    

        cyclone_field = hgt_data_global[t, lat - b : lat + b + 1, lon - b : lon + b + 1]
        
    low_result = low_center_test(cyclone_field, hgt_threshold, a, b) # now just modify this line
    # and function to incorporate the a and b parameters
    
    #print(low_result[0])
    
    if low_result[0]:
        
        raw_lows_z[low_center_count,0] = l
        y,m,d,h = date_from_netcdftime(domain_time[t], time_y0)
        raw_lows_z[low_center_count,1] = y
        
        raw_lows_z[low_center_count,2] = m
        raw_lows_z[low_center_count,3] = d
        raw_lows_z[low_center_count,4] = h
        raw_lows_z[low_center_count,5] = netcdf_time_scale(y,m,d,h,time_y0)
                
        raw_lows_z[low_center_count,6] = domain_lats[lat]
        raw_lows_z[low_center_count,7] = domain_lons[lon]
        
        raw_lows_z[low_center_count,8] = low_result[1]
        raw_lows_z[low_center_count,9] = low_result[2]
        raw_lows_z[low_center_count,10] = low_result[3]
        
        low_center_count = low_center_count + 1
    
    else:
        
        print('**** ERROR IN TRACKING SCHEME !!!!!! ****')
        print('tstep,lat,lon: ',t , ',', lat, ',', lon)
        
        continue
           
xr_obj.close()
   
#Remove empty rows from 'raw_lows' and save it in a file. 
ir = (raw_lows_z[:,1] != 0)
raw_lows_z = raw_lows_z[ir,:] 

#ADD THE OPTION TO REMOVE THE MINOR LOW CENTERS HERE
if filter_minor_lows:

    eliminate_secondary_lows(raw_lows_z, low_filtering_spatial_threshold)

output_filename = str(save_path) + 'rawlows_pg_' + str(p_level) + '_' + dataset + '.txt'
np.savetxt(output_filename, raw_lows_z, delimiter = ',') 

In [11]:
if filter_minor_lows:

    eliminate_secondary_lows(raw_lows_z, low_filtering_spatial_threshold)

output_filename = str(save_path) + 'rawlows_pg_' + str(p_level) + '_' + dataset + '.txt'
np.savetxt(output_filename, raw_lows_z, delimiter = ',') 